<a href="https://colab.research.google.com/github/manick2411/gemini-utils/blob/main/Function_Calling_Made_Easy_by_Vertex_Python_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Function Calling Made Easy by Vertex Python SDK**

In this notebook, Let us explore how Vertex AI Python SDK helps in making Function Calling very easy.



In [59]:
#Update Vertex AI Python SDK to latest version
!pip3 install google-cloud-aiplatform>=1.54.0 --upgrade

In [8]:
# Restart Kernel
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Authenticate to GCP with your credentials




In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

Enter the GCP Project ID and region which access to Google Gemini Models.

In [2]:
PROJECT_ID = "generative-ai-trials-427413"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

# Initiate Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

Create any function which you want the Gemini to use to do some action.

When Creating a function be mindful of following these steps. Its compoulsary to have these.
1. Function must have a **doc string** which describes what the function does,what arguments are needed and what each argument is expected to be as well as what the function returns.
2. Function declaration must have type hints.
3. Also follow more best practices mentioned here [Function Calling Vertex Documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling#best-practices)
Here we are giving Gemini the ability to get realtime current weather condition

In [3]:
import requests
from google.colab import userdata

def get_current_weather_data(city:str) -> str:
  """
  Gets weather in the specified location.
  Args:
      city: The city for which to get the weather as a str.
  Returns:
      Whole Weather data as a dictionary
  """
  url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={userdata.get('OPENWEATHER_API_KEY')}"
  response = requests.get(url)
  return response.json()["weather"][0]["description"]

Generally in Function Calling, Most Models SDK expect the function declaration in OpenAPI format which is time consuming and not easy to construct. And another head ache is parsing the response from the model to get the arguments to be executed.

This is where Vertex AI Python SDK helps developers.

In the next cell, We will see how Vertex SDK makes our life a little better by converting the function to the needed structure to pass as a tool to Gemini.

In [4]:
# Importing all the needed modules from the SDK
from vertexai.preview.generative_models import GenerativeModel,FunctionDeclaration,Tool,AutomaticFunctionCallingResponder

# Converting a python function directly to Gemini compatible Tool
get_current_weather_func = FunctionDeclaration.from_func(get_current_weather_data)
weather_tool = Tool(function_declarations=[get_current_weather_func],)

As mentioned earlier, The second head ache is getting the Arguments that Gemini wishes to pass and parsing it and then executing the function with the arguments and forming the execution result and passing it back to Gemini.

Vertex SDK takes care of all these with the help of AutomaticFunctionCallingResponder.

In [5]:
model = GenerativeModel("gemini-1.5-flash-001",tools=[weather_tool],) # Initialize the model with model name, and tools consisting of tools that we created
afc_responder = AutomaticFunctionCallingResponder(max_automatic_function_calls=5,) # Initialize AFC with maximum number of function calls allowed
chat = model.start_chat(responder=afc_responder) # Initialize a Chat with AFC as responder

Send Message to Gemini which triggers it to use our function.

When you run, You will just see the Output Response from Gemini. But as mentioned earlier all the Function Calling steps happen.

In [6]:
response1 = chat.send_message("I am from Mumbai. I am planning to go outside now. Should i carry a umbrella?")
print(response1.text)

It's overcast in Mumbai. You might want to carry an umbrella just in case. 



By getting the history of Chat, We can observe What Gemini has requested what our function returned and how Gemini interpreted it and used it to respond to end user needs.

In [7]:
chat.history

[role: "user"
 parts {
   text: "I am from Mumbai. I am planning to go outside now. Should i carry a umbrella?"
 },
 role: "model"
 parts {
   function_call {
     name: "get_current_weather_data"
     args {
       fields {
         key: "city"
         value {
           string_value: "Mumbai"
         }
       }
     }
   }
 },
 role: "user"
 parts {
   function_response {
     name: "get_current_weather_data"
     response {
       fields {
         key: "result"
         value {
           string_value: "overcast clouds"
         }
       }
     }
   }
 },
 role: "model"
 parts {
   text: "It\'s overcast in Mumbai. You might want to carry an umbrella just in case. \n"
 }]